# week 13
## 201710957 comp. sci. 이유진

블록체인에서 주문을 하고, 그 내역을 로컬 파일에 작성해 보자.

블록체인으로 주문을 전송하면, 이벤트가 발생하고 주문내역을 로컬파일에 쓰게 된다.

* EventTest.sol에서는 이벤트가 발생하면서 그 로그를 파일에 작성한다 (OrderEvent.txt)

* OrderEvent.sol을 수정하여:

	* order(상품항목, 주문 개수, 배송지) 함수 작성

	* getBalance() 함수 작성. 해당 프로그램 생성한 Owner만 읽을 수 있게 modfier isOnwer 작성.



## 1) Solidity 코딩

In [1]:
%%writefile C:\Users\sofod\Code\201710957\src\orderEvent_week13.sol
pragma solidity ^0.5.0;
contract orderEvent_week13{
    address owner;
    
    event OrderLog(address sender , bytes2 item_id , uint quantanty, string dest_addr);
    
    constructor() public {
        owner = msg.sender;
    }
    function order( bytes2 _itemId, uint quantanty , string memory dest_addr ) public payable {
        emit OrderLog(msg.sender, _itemId , quantanty, dest_addr);
    }
    modifier isOwner(){
        if(msg.sender != owner){
            revert();
        }
        _;
    }
    function getBalance() view public isOwner returns(uint){
        return address(this).balance;
    }
}

Overwriting C:\Users\sofod\Code\201710957\src\orderEvent_week13.sol


## 2) Solc를 이용하여 컴파일

In [2]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.0+commit.1d4f565a.Windows.msvc


In [3]:
!solc --abi --bin --gas C:\Users\sofod\Code\201710957\src\orderEvent_week13.sol


======= C:/Users/sofod/Code/201710957/src/orderEvent_week13.sol:orderEvent_week13 =======
Gas estimation:
construction:
   20474 + 165000 = 185474
external:
   getBalance():	883
   order(bytes2,uint256,string):	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610339806100606000396000f3fe60806040526004361061004c576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100515780637a59a9291461007c575b600080fd5b34801561005d57600080fd5b5061006661016c565b6040518082815260200191505060405180910390f35b61016a6004803603606081101561009257600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100e457600080fd5b8201836020820111156100f657600080fd5b8035906020019184600183028401116401000000008311171561011857600080fd5b91908080601f0160

## 3) node 를 이용한 배포 (ganache 사용)

In [20]:
%%writefile C:\Users\sofod\Code\201710957\src\orderEvent_week13_deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantanty","type":"uint256"},{"name":"dest_addr","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"sender","type":"address"},{"indexed":false,"name":"item_id","type":"bytes2"},{"indexed":false,"name":"quantanty","type":"uint256"},{"indexed":false,"name":"dest_addr","type":"string"}],"name":"OrderLog","type":"event"}];
var _bin="0x"+"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610339806100606000396000f3fe60806040526004361061004c576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100515780637a59a9291461007c575b600080fd5b34801561005d57600080fd5b5061006661016c565b6040518082815260200191505060405180910390f35b61016a6004803603606081101561009257600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100e457600080fd5b8201836020820111156100f657600080fd5b8035906020019184600183028401116401000000008311171561011857600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101e6565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156101c957600080fd5b3073ffffffffffffffffffffffffffffffffffffffff1631905090565b7f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a33848484604051808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001847dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102cb5780820151818401526020810190506102b0565b50505050905090810190601f1680156102f85780820380516001836020036101000a031916815260200191505b509550505050505060405180910390a150505056fea165627a7a7230582019e9c79ef3fdb61d13a82582cac2dc73bda0e3e5e49053ac427a4e0908c53f860029";



var orderContract = new web3.eth.Contract(_abiArray);
orderContract
    .deploy({
            data: _bin
    })
    .send({
     from: "0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc",
     gas: '1000000'
    }, function (error, transactionHash){ 
            console.log("error :" + error);
            console.log("hash : " + transactionHash); 
    })
    .then(function(newContractInstance){
        console.log("address : " + newContractInstance.options.address)
    });

Overwriting C:\Users\sofod\Code\201710957\src\orderEvent_week13_deploy.js


In [21]:
!node C:\Users\sofod\Code\201710957\src\orderEvent_week13_deploy.js

error :null
hash : 0x1599f65710661009ac66e7b36a352f1d0358c8846e416f905c7befb4f6418c66
address : 0x28130CD843D0150e4e9Dc56dD4078d508f45BDdC


## 4) 사용

### async 방식을 이용하여 event를 출력하였다.

In [22]:
%%writefile C:\Users\sofod\Code\201710957\src\orderEvent_week13_use.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://localhost:8345"));
}

var fs = require('fs');

var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_itemId","type":"bytes2"},{"name":"quantanty","type":"uint256"},{"name":"dest_addr","type":"string"}],"name":"order","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"sender","type":"address"},{"indexed":false,"name":"item_id","type":"bytes2"},{"indexed":false,"name":"quantanty","type":"uint256"},{"indexed":false,"name":"dest_addr","type":"string"}],"name":"OrderLog","type":"event"}];
var _bin="0x"+"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610339806100606000396000f3fe60806040526004361061004c576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100515780637a59a9291461007c575b600080fd5b34801561005d57600080fd5b5061006661016c565b6040518082815260200191505060405180910390f35b61016a6004803603606081101561009257600080fd5b8101908080357dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916906020019092919080359060200190929190803590602001906401000000008111156100e457600080fd5b8201836020820111156100f657600080fd5b8035906020019184600183028401116401000000008311171561011857600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f8201169050808301925050505050505091929192905050506101e6565b005b60008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156101c957600080fd5b3073ffffffffffffffffffffffffffffffffffffffff1631905090565b7f0cee52e22e6b387c4183ee502522fa33f030e118cc6065c7a842e8b7c00e4a6a33848484604051808573ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001847dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff19167dffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff1916815260200183815260200180602001828103825283818151815260200191508051906020019080838360005b838110156102cb5780820151818401526020810190506102b0565b50505050905090810190601f1680156102f85780820380516001836020036101000a031916815260200191505b509550505050505060405180910390a150505056fea165627a7a7230582019e9c79ef3fdb61d13a82582cac2dc73bda0e3e5e49053ac427a4e0908c53f860029";

async function doIt() {
    var _order = new web3.eth.Contract(_abiArray, '0x28130CD843D0150e4e9Dc56dD4078d508f45BDdC');
    var event = _order.events.OrderLog({fromBlock:0},function(error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues.sender)+","+  JSON.stringify(result.returnValues.item_id)
            + ","+JSON.stringify(result.returnValues.quantanty) +","+  JSON.stringify(result.returnValues.dest_addr)+"\n";
            console.log("---------------------------------------------");
            console.log("Event fired: " + log);
            fs.appendFile("OrderEvent.txt", log ,"utf-8",function(e) {
                if(!e){
                    console.log(">>Writing to file\n");
                }
            });
        }
    });
    
    const accounts = await web3.eth.getAccounts();

    // this will fire an event
    my = await _order.methods.order("0x1111", 3 , "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:30})
 
    console.log("---> Order called " + JSON.stringify(my.events.OrderLog.returnValues));
    
    my = await _order.methods.order("0x1111", 5 , "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:30})
 
    console.log("---> Order called " + JSON.stringify(my.events.OrderLog.returnValues));

    my = await _order.methods.order("0x1111", 20 , "20 2-gil Hongji-dong Jongro-gu Seoul")
        .send({from: accounts[0], gas: 100000, value:30})
 
    console.log("---> Order called " + JSON.stringify(my.events.OrderLog.returnValues));

    process.exit(1);

}

doIt();


Overwriting C:\Users\sofod\Code\201710957\src\orderEvent_week13_use.js


In [23]:
!node C:\Users\sofod\Code\201710957\src\orderEvent_week13_use.js

---------------------------------------------
Event fired: "0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","0x1111","3","20 2-gil Hongji-dong Jongro-gu Seoul"

>>Writing to file

---> Order called {"0":"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","1":"0x1111","2":"3","3":"20 2-gil Hongji-dong Jongro-gu Seoul","sender":"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","item_id":"0x1111","quantanty":"3","dest_addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---------------------------------------------
Event fired: "0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","0x1111","5","20 2-gil Hongji-dong Jongro-gu Seoul"

>>Writing to file

---> Order called {"0":"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","1":"0x1111","2":"5","3":"20 2-gil Hongji-dong Jongro-gu Seoul","sender":"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","item_id":"0x1111","quantanty":"5","dest_addr":"20 2-gil Hongji-dong Jongro-gu Seoul"}
---------------------------------------------
Event fired: "0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc

## 작성된 log내용 

In [24]:
!type OrderEvent.txt

"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","0x1111","3","20 2-gil Hongji-dong Jongro-gu Seoul"
"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","0x1111","5","20 2-gil Hongji-dong Jongro-gu Seoul"
"0x9E55bAE2D4E5cf459adc5170ac6277C9749FF3cc","0x1111","20","20 2-gil Hongji-dong Jongro-gu Seoul"
